# Transfer Learning with a Pre-trained CNN — Cats vs Dogs (MobileNetV2)

This notebook fulfills the assignment requirement:
**Write your own code to modify a pre-trained CNN model and apply suitable fine-tuning to fit your case study.**

**Case study:** Binary image classification (**Cat vs Dog**) using **MobileNetV2** pre-trained on ImageNet.


## 1) Environment Setup

Import required libraries and verify GPU availability (Google Colab recommended).

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt

print("TensorFlow:", tf.__version__)
print("GPUs:", tf.config.list_physical_devices("GPU"))

# (Optional) Enable memory growth on GPU
gpus = tf.config.list_physical_devices("GPU")
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("✅ GPU memory growth enabled")
    except Exception as e:
        print("⚠️ Could not set GPU memory growth:", e)


## 2) Dataset Preparation

We use the **cats_vs_dogs** dataset from TensorFlow Datasets (TFDS).
We create an 80/20 train/validation split and resize images to **224×224** to match MobileNetV2.

In [ ]:
(ds_train, ds_val), ds_info = tfds.load(
    "cats_vs_dogs",
    split=["train[:80%]", "train[80%:]"],
    as_supervised=True,
    with_info=True,
)
print(ds_info)


In [ ]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.1),
    ],
    name="data_augmentation",
)

def preprocess(image, label):
    image = tf.image.resize(image, IMG_SIZE)
    image = tf.cast(image, tf.float32)
    return image, label

ds_train = ds_train.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
ds_val   = ds_val.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)

ds_train = ds_train.shuffle(1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
ds_val   = ds_val.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)


### Sanity check: visualize samples

In [ ]:
class_names = ["cat", "dog"]  # 0 -> cat, 1 -> dog

for images, labels in ds_train.take(1):
    plt.figure(figsize=(8, 8))
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[int(labels[i].numpy())])
        plt.axis("off")
    plt.show()


## 3) Modify a Pre-trained CNN (MobileNetV2)

We load **MobileNetV2** (pre-trained on ImageNet) without its original classifier (`include_top=False`).
Then we add a new head for binary classification.

In [ ]:
base_model = keras.applications.MobileNetV2(
    input_shape=IMG_SIZE + (3,),
    include_top=False,
    weights="imagenet",
)

# Stage 1: feature extraction
base_model.trainable = False

inputs = keras.Input(shape=IMG_SIZE + (3,), name="input_image")
x = data_augmentation(inputs)
x = keras.applications.mobilenet_v2.preprocess_input(x)

x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D(name="gap")(x)
x = layers.Dropout(0.2, name="dropout")(x)
outputs = layers.Dense(1, activation="sigmoid", name="pred")(x)

model = keras.Model(inputs, outputs, name="mobilenetv2_cats_dogs")
model.summary()


## 4) Training — Stage 1 (Feature Extraction)

Train only the new head while the pre-trained base is frozen.

In [ ]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss="binary_crossentropy",
    metrics=["accuracy"],
)

history_stage1 = model.fit(
    ds_train,
    validation_data=ds_val,
    epochs=3,
)


### Plot accuracy (Stage 1)

In [ ]:
plt.figure()
plt.plot(history_stage1.history["accuracy"], label="train_acc")
plt.plot(history_stage1.history["val_accuracy"], label="val_acc")
plt.title("Stage 1 — Feature Extraction Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()
plt.show()


## 5) Fine-Tuning — Stage 2

Unfreeze part of the base model and continue training with a **smaller learning rate**.

In [ ]:
base_model.trainable = True

fine_tune_at = 100
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-5),
    loss="binary_crossentropy",
    metrics=["accuracy"],
)

history_stage2 = model.fit(
    ds_train,
    validation_data=ds_val,
    epochs=3,
)


### Plot accuracy (Stage 1 + Stage 2)

In [ ]:
acc = history_stage1.history["accuracy"] + history_stage2.history["accuracy"]
val_acc = history_stage1.history["val_accuracy"] + history_stage2.history["val_accuracy"]

plt.figure()
plt.plot(acc, label="train_acc")
plt.plot(val_acc, label="val_acc")
plt.axvline(x=len(history_stage1.history["accuracy"]) - 1, linestyle="--", label="Start Fine-Tuning")
plt.title("Accuracy Across Stages")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()
plt.show()


## 6) Evaluation

In [ ]:
loss, accuracy = model.evaluate(ds_val)
print(f"Validation Loss: {loss:.4f}")
print(f"Validation Accuracy: {accuracy:.4f}")


## 7) Inference Example

In [ ]:
for images, labels in ds_val.take(1):
    img = images[0]
    true_label = int(labels[0].numpy())
    break

img_batch = tf.expand_dims(img, axis=0)
img_batch = keras.applications.mobilenet_v2.preprocess_input(img_batch)

pred_prob = float(model.predict(img_batch)[0][0])
pred_label = 1 if pred_prob >= 0.5 else 0

plt.figure()
plt.imshow(img.numpy().astype("uint8"))
plt.title(f"Pred: {class_names[pred_label]} ({pred_prob:.2f}) | True: {class_names[true_label]}")
plt.axis("off")
plt.show()


## 8) Save Model

In [ ]:
model.save("cats_dogs_finetuned.h5")
print("✅ Saved: cats_dogs_finetuned.h5")


## Conclusion

You modified a pre-trained CNN (MobileNetV2) by replacing its top classifier, trained a new head, and then fine-tuned part of the network — matching the assignment requirements.